In [ ]:
%matplotlib inline
from IPython.display import clear_output

from srbot import *

In [ ]:
#generic slayer killer

#pyrefiends config
monster_colors = [[251,179,70],[251,139,90]]
monster_radius = 20
item_names = ['addy','sapphire','death rune','fire runes','chaos runes','coins','mith']
pickup_items = [[[89,113,91],[18,22,18]],[[40,40,153]],[[225,225,225],[22,18,18]],[[167,33,20]],[[223,167,26]],[[176,143,20]],[[66,66,96]]]
#item_names = ['addy','sapphire','death rune','runes / steel','fire runes','chaos runes','coins','mith']
#pickup_items = [[[89,113,91],[18,22,18]],[[40,40,153]],[[225,225,225],[22,18,18]],[[20,17,17],[149,139,148]],[[167,33,20]],[[223,167,26]],[[176,143,20]],[[66,66,96]]]

target()
total_kills = 0
logins = 0
last_fight = mark_time()
while True:
    client = get_client()
    if len(find_bitmap(loginscreen,client)) > 0:
        logins = logins + 1
        if logins > 10:
            raise RuntimeError('Too many logins! Bailing out before we get fishy!')
        login()
        continue
    if mark_time() - last_fight > 10*60:
        raise RuntimeError('No fights in a while, not good.')
            
    mainscreen = get_mainscreen()
    hp_r = find_colors([255,0,0],mainscreen,tol=0.01,mode='hsl') - [msxc-msxs,msyc-msys]
    hp_g = find_colors([0,255,0],mainscreen,tol=0.01,mode='hsl') - [msxc-msxs,msyc-msys]
    
    if len(np.concatenate([hp_r])) > 10: #in combat
        sleep(1.0)
        last_fight = mark_time()
        continue

    picked_up = False
    for i,colors in enumerate(pickup_items):
        mainscreen = get_mainscreen()
        found = None
        for color in colors:
            if np.std(color) < 15:
                tol = (1.0,0.08,0.08)
            else:
                tol = (0.05,0.08,0.08)
            ifound = find_colors(color,mainscreen,tol=tol,mode='hsl')
            if len(ifound):
                ifound = ifound[ifound[:,1]>20]
                if found is None:
                    found = ifound[ifound[:,1]>20]
                else:
                    found = filter_near(found,ifound,20)
            else:
                found = []
        found = filter_far(found,[msxc-msxs,msyc-msys],35)
        if found is not None and len(found) > 0:
            clusters,counts = cluster(found,20)
            print('found some',item_names[i],counts)
            found = [np.mean(cluster.T,axis=1) for cluster in clusters[np.logical_and(counts>6,counts<200)]]
            for point in found:
                move_mouse(*(point+[msxs,msys]))
                sleep(0.1)
                uptext = get_uptext()
                orangetxt = find_colors([225,129,60],uptext,tol=0.08,mode='hsl')
                if len(orangetxt) > 20:
                    click_mouse(*(point+[msxs,msys]))
                    flag_wait()
                    picked_up = True
                    break
        
    
    mainscreen = get_mainscreen()
    found = None
    for color in monster_colors:
        ifound = find_colors(color,mainscreen,tol=0.08,mode='hsl')
        if found is None:
            found = ifound
        else:
            found = filter_near(found,ifound,monster_radius)
        
    if len(found) > 0:
        np.random.shuffle(found)
        move_mouse(*(found[0]+[msxs,msys]))
        sleep(0.2)
        uptext = get_uptext()
        greentxt = find_colors([0,255,0],uptext,tol=0.2,mode='hsl')
        if len(greentxt) > 10:
            total_kills += 1
            print('Attacking number %i'%total_kills)
            click_mouse(*(found[0]+[msxs,msys]))
            sleep(10.0)
            continue